# **BUSINESS CASE 2: SIEMENS ADVANTA SALES FORECASTING**  


## 🎓 Master’s Program in Data Science & Advanced Analytics 
**Nova IMS** | March 2025   
**Course:** Business Cases with Data Science

## 👥 Team **Group A**  
- **Alice Viegas** | 20240572  
- **Bernardo Faria** | 20240579  
- **Dinis Pinto** | 20240612  
- **Daan van Holten** | 20240681
- **Philippe Dutranoit** | 20240518

## 📊 Goal of the notebook

in this notebook we make the build and test the models for forcasting

# Imports

In [83]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import itertools
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from prophet import Prophet


from Functions import *

import importlib
imported_module = importlib.import_module("Functions")
importlib.reload(imported_module)


<module 'Functions' from '/Users/philippedutranoit/document_ordi/nova/business_case_for_data_sciense/cases/BC_DA_Siemens/Notebooks/Functions.py'>

In [65]:
sales_agg1 = pd.read_csv('../data/sales_agg1.csv').set_index('DATE')
sales_agg3 = pd.read_csv('../data/sales_agg3.csv').set_index('DATE')
sales_agg4 = pd.read_csv('../data/sales_agg4.csv').set_index('DATE')
sales_agg5 = pd.read_csv('../data/sales_agg5.csv').set_index('DATE')
sales_agg6 = pd.read_csv('../data/sales_agg6.csv').set_index('DATE')
sales_agg8 = pd.read_csv('../data/sales_agg8.csv').set_index('DATE')
sales_agg9 = pd.read_csv('../data/sales_agg9.csv').set_index('DATE')
sales_agg11 = pd.read_csv('../data/sales_agg11.csv').set_index('DATE')
sales_agg12 = pd.read_csv('../data/sales_agg12.csv').set_index('DATE')
sales_agg13 = pd.read_csv('../data/sales_agg13.csv').set_index('DATE')
sales_agg14 = pd.read_csv('../data/sales_agg14.csv').set_index('DATE')
sales_agg16 = pd.read_csv('../data/sales_agg16.csv').set_index('DATE')
sales_agg20 = pd.read_csv('../data/sales_agg20.csv').set_index('DATE')
sales_agg36 = pd.read_csv('../data/sales_agg36.csv').set_index('DATE')

df_market = pd.read_csv('../data/df_market.csv').set_index('date')
test_set = pd.read_csv('../Data/Case2_Test Set Template.csv', sep=';')
print(sales_agg1.head())
df_market.head()



              Sales_EUR
DATE                   
2018-10-01  36098918.79
2018-11-01   5140760.00
2018-12-01  37889612.12
2019-01-01  27728148.35
2019-02-01  34793163.53


,MAB_ELE_PRO156,MAB_ELE_SHP156,MAB_ELE_PRO250,MAB_ELE_SHP250,MAB_ELE_PRO276,MAB_ELE_SHP276,MAB_ELE_PRO380,MAB_ELE_SHP380,MAB_ELE_PRO392,MAB_ELE_SHP392,...,PRO28250_org,PRO28276_org,PRO27840_org,PRO271000_org,PRO27756_org,PRO27826_org,PRO27380_org,PRO27392_org,PRO27250_org,PRO27276_org
date,,,,,,,,,,,,,,,,,,,,,
2004-02-01,16.940704,16.940704,112.091273,83.458866,82.623037,79.452532,124.289603,86.560493,109.334010,110.495272,...,118.274109,80.829010,117.723991,NaN,81.100000,120.706516,141.510864,106.161262,102.077057,85.913200
2004-03-01,23.711852,23.711852,136.327976,106.168192,100.556582,97.012918,143.411662,106.344544,140.884616,144.686166,...,148.121841,102.130104,119.220779,NaN,76.690307,138.309550,152.880234,140.288741,117.225685,97.670815
2004-04-01,24.435235,24.435235,117.791806,92.007646,89.653203,84.932358,129.083828,95.579673,105.853579,102.655769,...,125.482231,90.961426,117.441124,NaN,71.552403,115.557330,137.796875,106.271197,105.335777,87.253983
2004-05-01,23.708115,23.708115,109.002541,85.696486,86.880571,82.372794,135.590391,100.087039,101.864777,100.305285,...,116.649750,88.082901,117.899216,NaN,66.414500,119.269534,143.860535,101.608710,96.616508,84.675552
2004-06-01,27.009138,27.009138,133.785737,106.641482,99.010814,95.108740,136.424935,110.889719,120.332920,119.616380,...,143.248734,100.978699,119.499107,NaN,61.276596,128.849416,144.315308,116.655248,118.458710,95.401802


In [66]:
test_set

,Month Year,Mapped_GCK,Sales_EUR
0,Mai 22,#3,NaN
1,Jun 22,#3,NaN
2,Jul 22,#3,NaN
3,Aug 22,#3,NaN
4,Sep 22,#3,NaN
...,...,...,...
135,Okt 22,#1,NaN
136,Nov 22,#1,NaN
137,Dez 22,#1,NaN
138,Jan 23,#1,NaN


In [67]:
month_map = {
    "Jan": 1,
    "Feb": 2,
    "Mai": 3,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Okt": 10,
    "Nov": 11,
    "Dez": 12
}
def parse_month_year(date_string):
    month_abbr = date_string[:3]
    year_suffix = date_string[-2:]
    month_num = month_map.get(month_abbr)

    if not month_num:
        raise ValueError(f"Unknown month abbreviation: {month_abbr}")
    
    return datetime.strptime(f"{month_num} 01 {year_suffix}", "%m %d %y").date()

# Apply conversion safely
test_set['Month Year'] = test_set['Month Year'].apply(parse_month_year)

# preparation 

In [68]:
def train_val_split(df_target, df_market, target_column = "Sales_EUR" ,train_size = 0.8):

    """
    this function takes in two dataframes, one with the target variable and the other with the market data
    and returns the train and test sets for the target variable and the market data
    parameter:
        df_target: dataframe with the target variable
        df_market: dataframe with the market data
        train_size: the ratio of the train set to the entire dataset
    return:
        X_train: the train set for the market data
        X_test: the test set for the market data
        y_train: the train set for the target variable
        y_test: the test set for the target variable
    """
    df = pd.merge(df_target, df_market, left_index=True, right_index=True, how='inner')
    df.sort_index(inplace=True)
    split_index = int(len(df) * train_size)

    train = df.iloc[:split_index]
    val = df.iloc[split_index:]

    X_train = train.drop(target_column, axis=1)
    y_train = train[target_column]
    X_val = val.drop(target_column, axis=1)
    y_val = val[target_column]

    return X_train, X_val, y_train, y_val

In [69]:
def model_evaluation_XGboost(X_train, y_train, X_val, y_val, param_grid):
    """
    Performs manual grid search using validation set (not cross-validation).
    
    Parameters:
    - X_train, y_train: training data
    - X_val, y_val: validation data
    - param_grid: dictionary of hyperparameters

    Returns:
    - best_model: trained model with best params
    - best_params: dict of best hyperparameters
    - best_rmse: RMSE score on validation set
    """
    keys, values = zip(*param_grid.items())
    best_rmse = float('inf')
    best_params = None
    best_model = None

    for param_combination in itertools.product(*values):
        params = dict(zip(keys, param_combination))
        
        model = XGBRegressor(
            objective='reg:squarederror',
            random_state=42,
            **params
        )
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        print(f"Params: {params} => RMSE: {rmse:.4f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params
            best_model = model

    return best_model, best_params, best_rmse


In [84]:
def model_evaluation_prophet(X_train, y_train, X_val, y_val, param_grid):
    """
    Performs manual grid search using validation set (not cross-validation) for Prophet model.
    
    Parameters:
    - X_train: training data features (DataFrame)
    - y_train: training data target (Series)
    - X_val: validation data features (DataFrame)
    - y_val: validation data target (Series)
    - param_grid: dictionary of hyperparameters

    Returns:
    - best_model: trained model with best params
    - best_params: dict of best hyperparameters
    - best_rmse: RMSE score on validation set
    """
    # Combine X and y for Prophet
    train_df = pd.DataFrame({'ds': X_train.index, 'y': y_train.values})
    val_df = pd.DataFrame({'ds': X_val.index, 'y': y_val.values})

    keys, values = zip(*param_grid.items())
    best_rmse = float('inf')
    best_params = None
    best_model = None

    for param_combination in itertools.product(*values):
        params = dict(zip(keys, param_combination))
        
        model = Prophet(**params)
        
        model.fit(train_df)
        future = model.make_future_dataframe(periods=len(val_df), freq='D')
        forecast = model.predict(future)
        y_pred = forecast['yhat'][-len(val_df):].values
        y_true = val_df['y'].values
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))

        print(f"Params: {params} => RMSE: {rmse:.4f}")
        
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params
            best_model = model

    return best_model, best_params, best_rmse

In [77]:
## EXAMPLE 
# XGboost
# Drop columns that as null values

column_to_drop_for_test = ["MAB_ELE_SHP826", "PRI27826_org","MAB_ELE_SHP756","MAB_ELE_PRO756", "PRO27756_org", "MAB_ELE_SHP840"]
df_market.drop(column_to_drop_for_test, axis=1, inplace=True)

X_train, X_val, y_train, y_val = train_val_split(sales_agg1, df_market, train_size = 0.8)

X_val.info() 

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7, 1],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.1, 0.3],
    'reg_lambda': [0, 0.1, 0.3]
}

best_model, best_params, best_rmse = model_evaluation_XGboost(X_train, y_train, X_val, y_val, param_grid)

print("Best Params:", best_params)
print("Validation RMSE:", best_rmse)



KeyError: "['MAB_ELE_SHP826', 'PRI27826_org', 'MAB_ELE_SHP756', 'MAB_ELE_PRO756', 'PRO27756_org', 'MAB_ELE_SHP840'] not found in axis"

In [87]:
# EXAMPLE
# Prophet
X_train, X_val, y_train, y_val = train_val_split(sales_agg1, df_market, train_size=0.8)

param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'yearly_seasonality': [True, False],
    'weekly_seasonality': [True, False],
    'daily_seasonality': [True, False],
    'seasonality_prior_scale': [0.1, 1.0, 10.0],
    'changepoint_prior_scale': [0.01, 0.1, 0.5]
}

best_model, best_params, best_rmse = model_evaluation_prophet(X_train, y_train, X_val, y_val, param_grid)

print("Best Params:", best_params)
print("Validation RMSE:", best_rmse)

15:13:22 - cmdstanpy - INFO - Chain [1] start processing
15:13:23 - cmdstanpy - INFO - Chain [1] done processing
15:13:23 - cmdstanpy - INFO - Chain [1] start processing
15:13:23 - cmdstanpy - INFO - Chain [1] done processing
15:13:23 - cmdstanpy - INFO - Chain [1] start processing


Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.01} => RMSE: 3357351.5431
Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.1} => RMSE: 3356405.4135


15:13:23 - cmdstanpy - INFO - Chain [1] done processing
15:13:23 - cmdstanpy - INFO - Chain [1] start processing
15:13:23 - cmdstanpy - INFO - Chain [1] done processing
15:13:23 - cmdstanpy - INFO - Chain [1] start processing


Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.5} => RMSE: 3854203.3100
Params: {'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 1.0, 'changepoint_prior_scale': 0.01} => RMSE: 69941639.1351


15:13:23 - cmdstanpy - INFO - Chain [1] done processing
15:13:23 - cmdstanpy - INFO - Chain [1] start processing


...

Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.5} => RMSE: 3853678.4100
Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 1.0, 'changepoint_prior_scale': 0.01} => RMSE: 3259864.6103
Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 1.0, 'changepoint_prior_scale': 0.1} => RMSE: 3264371.3843


15:14:51 - cmdstanpy - INFO - Chain [1] done processing
15:14:51 - cmdstanpy - INFO - Chain [1] start processing
15:14:51 - cmdstanpy - INFO - Chain [1] done processing
15:14:51 - cmdstanpy - INFO - Chain [1] start processing
15:14:51 - cmdstanpy - INFO - Chain [1] done processing
15:14:51 - cmdstanpy - INFO - Chain [1] start processing


Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 1.0, 'changepoint_prior_scale': 0.5} => RMSE: 3853678.4100
Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 10.0, 'changepoint_prior_scale': 0.01} => RMSE: 3259864.6103
Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 10.0, 'changepoint_prior_scale': 0.1} => RMSE: 3264371.3843


15:14:51 - cmdstanpy - INFO - Chain [1] done processing


Params: {'seasonality_mode': 'multiplicative', 'yearly_seasonality': False, 'weekly_seasonality': False, 'daily_seasonality': False, 'seasonality_prior_scale': 10.0, 'changepoint_prior_scale': 0.5} => RMSE: 3853678.4100
Best Params: {'seasonality_mode': 'additive', 'yearly_seasonality': False, 'weekly_seasonality': True, 'daily_seasonality': True, 'seasonality_prior_scale': 0.1, 'changepoint_prior_scale': 0.1}
Validation RMSE: 2866894.4721454247
